In [2]:
import matplotlib.pyplot as plt
from matplotlib import rcParams,rc
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import numpy as np
import xarray as xr
import sys
import os
from ncdf_io import ncdf_io


In [3]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2

In [4]:
base=os.getcwd()

In [5]:
dir1='/Users/joedhsu/Research/Rsync/Data/Mask/ETOPO/NOAA/'
file1='ETOPO1_Ice_c_gmt4.grd'


---

## Read the Topo file 

In [6]:
# ncdf_io(dir1+file1+'.nc')
ds=xr.open_dataset(dir1+file1+'.nc')

In [ ]:
ds['z'].plot()

**Changing the longitude from -180 to 180**

In [ ]:
longitude=ds['z'].x.values.copy()   
longitude[np.where(longitude<0)]=longitude[np.where(longitude<0)]+360.
ds['z'].x.values=longitude

---
## Read the Ocean mass 

In [ ]:
data1='./data/jpl_mascon_ocean-slf.reg.nc'
data2='./data/jpl_mascon_ocean.reg.nc'
ds_jpl_slfcorr=xr.open_dataset(data1)
ds_jpl=xr.open_dataset(data2)

In [ ]:
ds_jpl_slfcorr

In [ ]:
ds_jpl

**Interpolate the topo to match with the ocean mass data**

In [ ]:
ds_topo_interp=ds['z'].interp(coords={'x':ds_jpl_slfcorr.lon, 'y':ds_jpl_slfcorr.lat})

In [ ]:
ds_topo_interp.plot()

In [ ]:
ds_topo_interp.to_netcdf('./data/topo_interp.nc')

In [ ]:
ds_topo_interp=ds_topo_interp.drop(['x','y'])

In [ ]:
ds_topo_interp


---
**Plotting with Projection change**

`ccrs.Orthographic` projection does not seems to work for the interp topo result. Worng topo values (contourf cannot show value lower than -2000)  are shown for the whole map

In [ ]:
# set the figure and data projection
data_proj=ccrs.PlateCarree()
fig_proj_test=ccrs.Orthographic(0, 90)    # rotating the center of the projection  => Not working for topo
fig_proj=ccrs.NorthPolarStereo(0, 90)    # rotating the center of the projection

# set the figure axes
fig=plt.figure(facecolor='w',edgecolor='w')
ax1=fig.add_axes([0,0,1,1],projection=fig_proj)
ax2=fig.add_axes([1,0,1,1],projection=fig_proj_test)

# plot the data in each axes
ax1.set_title('Topo')
ax1.set_extent([-180,180,57,90], crs=data_proj)
ds_topo_interp.plot.contourf(ax=ax1, transform=data_proj, cmap='hot', levels=np.linspace(-4000,0,5))
ax1.coastlines(resolution='110m',linewidths=0.3)

ax2.set_extent([-180,180,57,90], crs=data_proj)
ds_topo_interp.plot.contourf(ax=ax2, transform=data_proj, cmap='hot', levels=np.linspace(-4000,0,5))
ax2.coastlines(resolution='110m',linewidths=0.3)

In [ ]:
# set the figure and data projection
data_proj=ccrs.PlateCarree()
fig_proj=ccrs.NorthPolarStereo(0, 90)   

# set the figure axes
hmm=70.    # mm
vmm=65.    # mm
fontsize=6
rcParams['xtick.direction'] = 'out'
rcParams['ytick.direction'] = 'out'
rcParams.update({'font.size': fontsize})
rc('font', family='sans-serif')
rc('font', serif='Helvetica')
mm2inch=0.0393701
hmm=hmm*mm2inch
vmm=vmm*mm2inch
fig=plt.figure(figsize=(hmm,vmm),dpi=300,facecolor='w',edgecolor='w')
# ax1=fig.add_axes([0.35,0.0,1.20,1],projection=fig_proj)
# ax2=fig.add_axes([1.2,0,1,1],projection=fig_proj)
ax1=fig.add_axes([0.25,0.065,1.05,0.87],projection=fig_proj)
ax2=fig.add_axes([1.2,0,1,1],projection=fig_proj)

# plot the data in each axes
levels=np.linspace(-0.1,0.8,10)
plot1=ds_jpl_slfcorr['beta'].sel(reg_variate='lin').plot.contourf(ax=ax1, transform=data_proj
        , levels=levels,cmap='gist_rainbow_r',add_colorbar=False, add_labels=True)
ds_topo_interp.plot.contour(ax=ax1, transform=data_proj
                            , colors='k', levels=[-3000],linewidths=0.5,linestyles='dashdot')
ds_topo_interp.plot.contour(ax=ax1, transform=data_proj
                            , colors='b', levels=[-2000],linewidths=0.5,linestyles='dashed')
ax1.set_title('JPL with SLF correction trend')
# ax1.text(fig_proj.transform_point(180,45,data_proj), 'NoSLFcorr')
ax1.set_extent([-180,180,57,90], crs=data_proj)
ax1.coastlines(resolution='110m',linewidths=0.8)


# plot2=ds_jpl['beta'].sel(reg_variate='lin').plot.contourf(ax=ax2, transform=data_proj
#         , levels=levels, extend='both',cmap='gist_rainbow_r'
#         , cbar_kwargs={'ticks': levels,'label':'cmH2O/yr'})
# ax2.set_title('JPL trend')
# # ax2.set_extent([-180,180,57,90], crs=data_proj)
# ds_topo_interp.plot.contour(ax=ax2, transform=data_proj
#                             , colors='k', levels=[-3000],linewidths=0.5,linestyles='dashdot')
# ds_topo_interp.plot.contour(ax=ax2, transform=data_proj
#                             , colors='b', levels=[-2000],linewidths=0.5,linestyles='dashed')
# ax2.coastlines(resolution='110m',linewidths=0.8)


####################  Grid on  ######################
# *must* call draw in order to get the axis boundary used to add ticks:
fig.canvas.draw()
xtick=np.linspace(0,360,37)-180.  # need to be -180~180
ytick=np.linspace(55,85,7)

#Label the end-points of the gridlines using the custom tick makers:
ax1.gridlines(xlocs=xtick, ylocs=ytick,color='gray', linestyle='--', linewidth=0.5)
ax1.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
ax1.yaxis.set_major_formatter(LATITUDE_FORMATTER)
# lambert_xticks(ax1, xtick)
# lambert_yticks(ax1, ytick)
ax2.gridlines(xlocs=xtick, ylocs=ytick,color='gray', linestyle='--', linewidth=0.5)
ax2.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
ax2.yaxis.set_major_formatter(LATITUDE_FORMATTER)

# # export the figure
# plt.savefig('./figures/arctic_jpl_mascon.pdf', dpi=300, facecolor='w', edgecolor='w',
#         orientation='landscape', papertype=None, format=None,
#         transparent=False, bbox_inches="tight", pad_inches=None,
#         frameon=None)



In [ ]:
# set the figure and data projection
data_proj=ccrs.PlateCarree()
fig_proj=ccrs.Orthographic(-0, 90)    # rotating the center of the projection
#fig_proj = ccrs.LambertConformal(central_longitude=-50, central_latitude=45)
# fig_proj = ccrs.Mercator(central_longitude=0.0, min_latitude=-80.0, max_latitude=84.0)

# set the figure axes
hmm=70.    # mm
vmm=65.    # mm
fontsize=6
rcParams['xtick.direction'] = 'out'
rcParams['ytick.direction'] = 'out'
rcParams.update({'font.size': fontsize})
rc('font', family='sans-serif')
rc('font', serif='Helvetica')
mm2inch=0.0393701
hmm=hmm*mm2inch
vmm=vmm*mm2inch
fig=plt.figure(dpi=300,facecolor='w',edgecolor='w')
ax1=fig.add_axes([0.25,0.07,1.05,0.86],projection=fig_proj)
ax2=fig.add_axes([1.2,0,1,1],projection=fig_proj)

# plot the data in each axes
# levels=np.linspace(-0.1,0.8,10)
# plot1=ds_jpl_slf['beta'].sel(reg_variate='lin').plot.contourf(ax=ax1, transform=data_proj
#         , levels=levels, extend='both',cmap='gist_rainbow_r'
#         , cbar_kwargs={'ticks': levels,'label':'cmH2O'})
ax1.set_title('JPL with SLF correction trend')
ax1.set_extent([-180,180,57,90], crs=data_proj)
# ds_topo_interp.plot.contour(ax=ax1, transform=data_proj
#                             , colors='k', levels=[-4000],linewidths=0.5,linestyles='dashdot')
# ds_topo_interp.plot.contourf(ax=ax1, transform=data_proj
#                             , colors='k', levels=[-2000],linewidths=0.5,linestyles='dashed')
ds_topo_interp.plot.contourf(ax=ax1, transform=data_proj
                            , colors='hot', levels=np.linspace(-4000,0,5))

# ds['z'].plot.contourf(ax=ax2, transform=data_proj
#                             , colors='hot', levels=np.linspace(-4000,0,5))
ax1.coastlines(resolution='110m',linewidths=0.3)
# ax2.coastlines(resolution='110m',linewidths=0.3)
#         , levels=levels, extend='both',cmap='gist_rainbow_r'
#         , cbar_kwargs{'ticks': levels,'label':'cmH2O'})


####################  Grid on  ######################
# *must* call draw in order to get the axis boundary used to add ticks:
fig.canvas.draw()
xtick=np.linspace(0,360,37)-180.  # need to be -180~180
ytick=np.linspace(55,85,7)

# Label the end-points of the gridlines using the custom tick makers:
# ax1.gridlines(xlocs=xtick, ylocs=ytick,color='gray', linestyle='--', linewidth=0.5)
# ax1.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
# ax1.yaxis.set_major_formatter(LATITUDE_FORMATTER)
# lambert_xticks(ax1, xtick)
# lambert_yticks(ax1, ytick)
# ax2.gridlines(xlocs=xtick, ylocs=ytick,color='gray', linestyle='--', linewidth=0.5)
# ax2.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
# ax2.yaxis.set_major_formatter(LATITUDE_FORMATTER)

# # export the figure
# plt.savefig('./figures/arctic_jpl_mascon.pdf', dpi=300, facecolor='w', edgecolor='w',
#         orientation='landscape', papertype=None, format=None,
#         transparent=False, bbox_inches="tight", pad_inches=None,
#         frameon=None)



In [ ]:
# set the figure and data projection
data_proj=ccrs.PlateCarree()
fig_proj=ccrs.Orthographic(-0, 90)    # rotating the center of the projection
#fig_proj = ccrs.LambertConformal(central_longitude=-50, central_latitude=45)
# fig_proj = ccrs.Mercator(central_longitude=0.0, min_latitude=-80.0, max_latitude=84.0)

# set the figure axes
hmm=70.    # mm
vmm=65.    # mm
fontsize=6
rcParams['xtick.direction'] = 'out'
rcParams['ytick.direction'] = 'out'
rcParams.update({'font.size': fontsize})
rc('font', family='sans-serif')
rc('font', serif='Helvetica')
mm2inch=0.0393701
hmm=hmm*mm2inch
vmm=vmm*mm2inch
fig=plt.figure(dpi=300,facecolor='w',edgecolor='w')
ax1=fig.add_axes([0.25,0.07,1.05,0.86],projection=fig_proj)
# ax2=fig.add_axes([1.2,0,1,1],projection=fig_proj)

# plot the data in each axes
# levels=np.linspace(-0.1,0.8,10)
# plot1=ds_jpl_slf['beta'].sel(reg_variate='lin').plot.contourf(ax=ax1, transform=data_proj
#         , levels=levels, extend='both',cmap='gist_rainbow_r'
#         , cbar_kwargs={'ticks': levels,'label':'cmH2O'})
ax1.set_title('JPL with SLF correction trend')
ax1.set_extent([-180,180,57,90], crs=data_proj)
# ds_topo_interp.plot.contour(ax=ax1, transform=data_proj
#                             , colors='k', levels=[-4000],linewidths=0.5,linestyles='dashdot')
# ds_topo_interp.plot.contourf(ax=ax1, transform=data_proj
#                             , colors='k', levels=[-2000],linewidths=0.5,linestyles='dashed')
ds_topo_interp.plot.contourf(ax=ax1, transform=data_proj
                            , colors='hot', levels=np.linspace(-4000,0,5))

ax1.coastlines(resolution='110m',linewidths=0.3)
#         , levels=levels, extend='both',cmap='gist_rainbow_r'
#         , cbar_kwargs{'ticks': levels,'label':'cmH2O'})


####################  Grid on  ######################
# *must* call draw in order to get the axis boundary used to add ticks:
fig.canvas.draw()
xtick=np.linspace(0,360,37)-180.  # need to be -180~180
ytick=np.linspace(55,85,7)

# Label the end-points of the gridlines using the custom tick makers:
# ax1.gridlines(xlocs=xtick, ylocs=ytick,color='gray', linestyle='--', linewidth=0.5)
# ax1.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
# ax1.yaxis.set_major_formatter(LATITUDE_FORMATTER)
# lambert_xticks(ax1, xtick)
# lambert_yticks(ax1, ytick)
# ax2.gridlines(xlocs=xtick, ylocs=ytick,color='gray', linestyle='--', linewidth=0.5)
# ax2.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
# ax2.yaxis.set_major_formatter(LATITUDE_FORMATTER)

# # export the figure
# plt.savefig('./figures/arctic_jpl_mascon.pdf', dpi=300, facecolor='w', edgecolor='w',
#         orientation='landscape', papertype=None, format=None,
#         transparent=False, bbox_inches="tight", pad_inches=None,
#         frameon=None)

